Copyright (c) Snap Inc. 2020. This sample code is made available by Snap Inc. for informational purposes only. It is provided as-is, without warranty of any kind, express or implied, including any warranties of merchantability, fitness for a particular purpose, or non-infringement. In no event will Snap Inc. be liable for any damages arising from the sample code or your use thereof.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import hashlib
import spacy
import os
import re
import json
from collections import OrderedDict
from operator import itemgetter
from spacy.lang.en.stop_words import STOP_WORDS
import string
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from xgboost import XGBClassifier
import sys
sys.path.append('../')

from tf_idf_vectorizer import *
from utils.snap_preprocessed_df_handle import *
from utils.EstimatorSelectionHelper import EstimatorSelectionHelper
from utils.classifier_setup import *

# BERT Classification
from sentence_transformers import SentenceTransformer
from scipy.stats import pearsonr, spearmanr
import pickle

In [ ]:
TRAIN_PATH = '../../data/dataframes/df_unique_with_similarity.pkl'
TEST_PATH = '../../data/dataframes/df_test_unique_with_similarity.pkl'

In [ ]:
def prepare_dataframe_tf_idf(PATH):
    df_with_keywords = get_dataframe(PATH)
    return df_with_keywords

In [ ]:
train_df = prepare_dataframe_tf_idf(TRAIN_PATH)

In [ ]:
test_df = prepare_dataframe_tf_idf(TEST_PATH)

In [ ]:
train_df.drop(columns=['k1','k2','textrank_similarity'], inplace=True)
test_df.drop(columns=['k1','k2','textrank_similarity'], inplace=True)

In [ ]:
train_df.columns

# Setup

In [ ]:
# model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

In [ ]:
articles_train = pd.concat([train_df[['id1','Input.article1']].\
                            rename(columns={'id1':'id','Input.article1':'article'}), \
                            train_df[['id2','Input.article2']].\
                            rename(columns={'id2':'id','Input.article2':'article'})]
                          ).drop_duplicates().reset_index(drop=True)
non_dup_articles_train = articles_train['id'].drop_duplicates().index
articles_train = articles_train.loc[non_dup_articles_train].reset_index(drop=True)

articles_test = pd.concat([test_df[['id1','Input.article1']].\
                            rename(columns={'id1':'id','Input.article1':'article'}), \
                            test_df[['id2','Input.article2']].\
                            rename(columns={'id2':'id','Input.article2':'article'})]
                          ).drop_duplicates().reset_index(drop=True)

non_dup_articles_test = articles_test['id'].drop_duplicates().index
articles_test = articles_test.loc[non_dup_articles_test].reset_index(drop=True)

In [ ]:
# sentence_embeddings_train = model.encode(articles_train['article'])

In [ ]:
# sentence_embeddings_test = model.encode(articles_test['article'])

In [ ]:
# with open('../../data/dataframes/roberta_sentence_embeddings_train.pkl', 'wb') as f:
#     pickle.dump(sentence_embeddings_train, f)

In [ ]:
# with open('../../data/dataframes/roberta_sentence_embeddings_test.pkl', 'wb') as f:
#     pickle.dump(sentence_embeddings_test, f)

In [ ]:
with open('../../data/dataframes/roberta_sentence_embeddings_train.pkl', 'rb') as f:
    sentence_embeddings_train = pickle.load(f)

In [ ]:
with open('../../data/dataframes/roberta_sentence_embeddings_test.pkl', 'rb') as f:
    sentence_embeddings_test = pickle.load(f)

In [ ]:
articles_train['roberta_embedding'] = sentence_embeddings_train
articles_test['roberta_embedding'] = sentence_embeddings_test

In [ ]:
train_df['roberta_embedding1'] = train_df['id1'].\
                                apply(lambda x: articles_train[articles_train['id']==x]\
                                      ['roberta_embedding'].values[0])
train_df['roberta_embedding2'] = train_df['id2'].\
                                apply(lambda x: articles_train[articles_train['id']==x]\
                                      ['roberta_embedding'].values[0])
train_df['roberta_similarity'] = train_df[['roberta_embedding1','roberta_embedding2']]\
                                        .apply(lambda row: \
                                               cosine_similarity(row['roberta_embedding1'].reshape(1, -1),\
                                                                 row['roberta_embedding2'].reshape(1, -1))[0][0], axis=1)

In [ ]:
test_df['roberta_embedding1'] = test_df['id1'].\
                                apply(lambda x: articles_test[articles_test['id']==x]\
                                      ['roberta_embedding'].values[0])
test_df['roberta_embedding2'] = test_df['id2'].\
                                apply(lambda x: articles_test[articles_test['id']==x]\
                                      ['roberta_embedding'].values[0])
test_df['roberta_similarity'] = test_df[['roberta_embedding1','roberta_embedding2']]\
                                        .apply(lambda row: \
                                               cosine_similarity(row['roberta_embedding1'].reshape(1, -1),\
                                                                 row['roberta_embedding2'].reshape(1, -1))[0][0], axis=1)

# Event Similarity

In [ ]:
models = {
           "XGBoost" : XGBClassifier()
}

params = {'XGBoost':  {"colsample_bytree": [0.3,0.5,0.8,1],"gamma":[0,10,50,100],
                        "max_depth": [2,4,6], # default 3\
                        "n_estimators": [50,100], # default 100
                        "subsample": [0.3,0.5,0.8,1]}
}

def custom_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

In [ ]:
helper_event = EstimatorSelectionHelper(models, params)

In [ ]:
helper_event.fit(train_df['roberta_similarity'].values.reshape(-1, 1),
            train_df['majority_same_event'],
            cv = 5,
            scoring=make_scorer(custom_scorer, greater_is_better=True), n_jobs=16, refit=True)

In [ ]:
helper_event.summary(test_df['roberta_similarity'], test_df['majority_same_event'])

# Topic Similarity

In [ ]:
models = {
           "XGBoost" : XGBClassifier()
}

params = {'XGBoost':  {"colsample_bytree": [0.3,0.5,0.8,1],"gamma":[0,10,50,100],
                        "max_depth": [2,4,6], # default 3\
                        "n_estimators": [50,100], # default 100
                        "subsample": [0.3,0.5,0.8,1]}
}

def custom_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

In [ ]:
train_df['majority_topic_1'] = train_df[train_df.columns[13:20]].idxmax(axis=1).str.split(".").str.get(-1)
train_df['majority_topic_2'] = train_df[train_df.columns[20:27]].idxmax(axis=1).str.split(".").str.get(-1)
train_df['majority_same_topic']=train_df['majority_topic_1'] == train_df['majority_topic_2']

In [ ]:
test_df['majority_topic_1'] = test_df[test_df.columns[13:20]].idxmax(axis=1).str.split(".").str.get(-1)
test_df['majority_topic_2'] = test_df[test_df.columns[20:27]].idxmax(axis=1).str.split(".").str.get(-1)
test_df['majority_same_topic']=test_df['majority_topic_1'] == test_df['majority_topic_2']

In [ ]:
helper_topic = EstimatorSelectionHelper(models, params)

In [ ]:
helper_topic.fit(train_df['roberta_similarity'].values.reshape(-1, 1),
            train_df['majority_same_topic'],
            cv = 5,
            scoring=make_scorer(custom_scorer, greater_is_better=True), n_jobs=16, refit=True)

In [ ]:
helper_topic.summary(test_df['roberta_similarity'], test_df['majority_same_topic'])